# Flag Codes from Tesselations of Manifolds

This notebook illustrates the construction of flag codes from tilings of manifolds as described on page 3 of  Quantum Pin Codes by Vuillot and Breuckmann.

In this approach, the starting point is a cell complex corresponding to a tiling of a manifold.

Flags are created by selecting a sequence of cells of each dimension in which a the cell of dimension $c$ is contained in the cell of dimension $c+1$. In the flag graph, flags are $c$-adjacent if they differ only by a cell of dimension $c$. Quantum codes are constructed by identifying checks with connected components of the flag graph where flags are $c_i$-adjacent for some $c_i$ in a set of 'colours' $\mathbf{c} := \{c_0,..,c_{d-1}\}$.

The dimension of the subgraph is the number of colours $d$ in $\mathbf{c}$. We generally  associate Z-checks with 2-dimensional subgraphs and  X-checks with D-dimensional subgraphs where $D$ is the dimension of the manifold.

Users can select from a variety of 2D and 3D cell complexes which have been generated using a Gap script, and explore the effect of various types of edge-contractions to reduce the number of physical qubits.

In [20]:
from add_parent_dir import *
from common import *
from NHow import *
from CSSLO import *
from flag_codes import *

def GExplore(A):
    c,n = A.shape
    todo = set(range(c))
    cycles = set()
    PT = dict()
    while len(todo) > 0:
        i = min(todo)
        todo.remove(i)
        PT[i] = None
        Rtodo = [i]
        while len(Rtodo) > 0:
            i = Rtodo.pop()
            for c in bin2Set(A[i]):
                for r in bin2Set(A.T[c]):
                    if r > i:
                        # print(i,r,PT)
                        if r not in PT:
                            Rtodo.append(r)
                            todo.remove(r)
                            PT[r] = i
                        else:
                            cycles.add((i,r))
    return PT,cycles

def Gpath2root(PT,i):
    mypath = [i]
    while i is not None:
        i = PT[i]
        mypath.append(i)
    mypath.reverse()
    return mypath

def Gpath2cycle(p1,p2):
    # print(func_name(),p1,p2)
    i = 0
    nMin = min(len(p1),len(p2))
    while i < nMin and ((p1[i] is None) or (p1[i] == p2[i])):
        i +=1
    p = p1[i:] + p2[i:]
    # print(func_name(),p)
    if i > 0:
        p.append(p1[i-1])
    return p

def Gcycles(A):
    C = []
    PT,cycles = GExplore(A)
    # print('Cycles',cycles)
    for (i,j) in cycles:
        p1,p2 = Gpath2root(PT,i),Gpath2root(PT,j)
        p = Gpath2cycle(p1,p2)
        C.append(p)
    return C

def expander42(n):
    s = (n-1)//2
    r = n // 2
    Rn = repCode(n)
    In = ZMatI(n)
    Inr = np.roll(ZMatI(n),shift=s,axis=-1)
    Ins = np.roll(ZMatI(n),shift=-s,axis=1)
    zeron = ZMatZeros((n,n))
    R1 = ZMatHstack([Rn,In,In,zeron])
    R2 = ZMatHstack([zeron,Ins,Inr,Rn])
    return ZMatVstack([R1,R2])

##########################################################
## Reflection Group Codes
##########################################################

## 2D hyperbolic tesselations
# myfile = "RG-4-6.txt"
myfile = "RG-4-5.txt"
# myfile = "RG-5-5.txt"
# myfile = "RG-7-7.txt"

codeList = importRGList(myfile)
# for i in range(len(codeList)):
#     A = codeList[i][1][1]
#     # print('A')
#     # print(ZMatPrint(A))
#     C = Gcycles(A)
#     cLen = [len(c) for c in C]
#     print(f'{i} Graph Cycle Structure',freqTablePrint(cLen))

# for i in range(1,5):
#     A = expander42(i)
#     # print('A')
#     # print(ZMatPrint(A))
#     C = Gcycles(A)
#     cLen = [len(c) for c in C]
#     # print('A')
#     # print(ZMatPrint(A))
#     print(f'{i} Graph Cycle Structure',freqTablePrint(cLen))
#     # for c in C:
#     #     print(c)

print('\n#########################################################')
print(f'## 2D Hyperbolic Tesselations {myfile}')
print(printRGList(codeList,myfile,checkValid=True))
i = 2
complex2D =  codeList[i][1][1:]
SX, SZ = complex2SurfaceCode(codeList[i][1])
SX,LX,SZ,LZ = CSSCode(SX=SX,SZ=SZ)
k,n = LX.shape
b,L = distGenetic(LZ,SZ,tB=1)
dZ = min(np.sum(L,axis=-1))
b,L = distGenetic(LX,SX,tB=1)
dX = min(np.sum(L,axis=-1))
d = min(dZ,dX)
print(f'2D Code: [[{n},{k},{d}]]')

j = 2
expander = expander42(j)
# expander = codeList[j][1][1]
print('Expander')
print(ZMatPrint(expander))
C = Gcycles(expander)
cLen = [len(c) for c in C]
print(f'Cycle Structure',freqTablePrint(cLen))

C = complexTCProd(complex2D,[expander])

print('\n#########################################################')
print(f'## Analysing Code {i}')
## Both SX and SZ are maximal subgraphs (as in Pin Codes paper)
print('\n#########################################################')
constrFun = Complex_to_Code_Mixed
print('\n#########################################################')
print('## Mixed Construction - No Edge Contraction')
res = analyseFlagCode(C,constrFun,calc_dist=False,coloured_logical_paulis=False,calc_LO=False)
print(res)
print('\n#########################################################')
# constrFun = Edge_Contraction_0
# print('\n#########################################################')
# print('## Mixed Construction -  Edge Contraction 0')
# res = analyseFlagCode(C,constrFun,calc_dist=False,coloured_logical_paulis=False,calc_LO=False)
# print(res)
# print('\n#########################################################')
constrFun = Edge_Contraction_3
print('\n#########################################################')
print('## Mixed Construction -  Edge Contraction 0')
res = analyseFlagCode(C,constrFun,calc_dist=False,coloured_logical_paulis=False,calc_LO=False)
print(res)
print('\n#########################################################')
constrFun = Complex_to_Code_MSG
print('\n#########################################################')
print('## MSG Construction - No Edge Contraction')
res = analyseFlagCode(C,constrFun,calc_dist=False,coloured_logical_paulis=False,calc_LO=False)
print(res)


#########################################################
## 2D Hyperbolic Tesselations RG-4-5.txt
Codes in File RG-4-5.txt:

i	index	Valid	|C0|	|C1|	|C2|
0	10	True	5	5	1
1	20	True	5	5	2
2	120	True	15	30	12
3	160	True	20	40	16
4	240	True	30	60	24
5	320	True	40	80	32
6	640	True	80	160	64
7	720	True	90	180	72
8	1200	True	150	300	120
9	1320	True	165	330	132
10	1320	True	165	330	132
11	1440	True	180	360	144
12	1920	True	240	480	192
13	2640	True	330	660	264
14	2640	True	330	660	264
15	3840	True	480	960	384
16	5120	True	640	1280	512
2D Code: [[30,5,3]]
Expander
11101000
11010100
00101011
00010111
Cycle Structure 2:3,4:1

#########################################################
## Analysing Code 2

#########################################################

#########################################################
## Mixed Construction - No Edge Contraction

Weights of stabiliser generators and logical Paulis:
SX 16:360,32:120,40:48,48:180,60:96
SZ 4:4080,6:960,8:1680,10:192
LX 144:2,300:2,3